In [185]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import featurewiz as gwiz

In [188]:
data = pd.read_csv('dataset.csv')
data.head()

,ID,Name,InChI,InChIKey,SMILES,Solubility,SD,Ocurrences,Group,MolWt,...,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT
0,A-3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1,G1,392.510,...,17.0,142.0,0.0,0.0,0.0,0.0,0.00,158.520601,0.000000e+00,210.377334
1,A-4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1,G1,169.183,...,0.0,62.0,2.0,0.0,1.0,3.0,29.10,75.183563,2.582996e+00,511.229248
2,A-5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1,G1,140.569,...,1.0,46.0,1.0,0.0,0.0,1.0,17.07,58.261134,3.009782e+00,202.661065
3,A-8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.0,1,G1,756.226,...,10.0,264.0,6.0,0.0,0.0,6.0,120.72,323.755434,2.322963e-07,1964.648666
4,A-9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1,G1,422.525,...,12.0,164.0,2.0,4.0,4.0,6.0,56.60,183.183268,1.084427e+00,769.899934


data.SMILES.count()
data.Solubility.count()
Chem.MolFromSmiles(data.SMILES[0])
Chem.MolFromSmiles('ClCC(Cl)(Cl)Cl')
m = Chem.MolFromSmiles('ClCC(Cl)(Cl)Cl')
m.GetNumAtoms()

HELP STRIPPING STRINGS?

In [190]:
df_obj = data.select_dtypes(['string'])
print (df_obj)

data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
print (data)

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[9982 rows x 0 columns]
        ID                                               Name   
0      A-3         N,N,N-trimethyloctadecan-1-aminium bromide  \
1      A-4                           Benzo[cd]indol-2(1H)-one   
2      A-5                               4-chlorobenzaldehyde   
3      A-8  zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...   
4      A-9  4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...   
...    ...                                                ...   
9977  I-84                                         tetracaine   
9978  I-85              

In [191]:
mol_list= []
for element in data.SMILES:
  mol = Chem.MolFromSmiles(element)
  mol_list.append(mol)

[02:13:06] WARNING: not removing hydrogen atom without neighbors
[02:13:06] WARNING: not removing hydrogen atom without neighbors
[02:13:06] WARNING: not removing hydrogen atom without neighbors
[02:13:06] WARNING: not removing hydrogen atom without neighbors
[02:13:06] WARNING: not removing hydrogen atom without neighbors
[02:13:07] WARNING: not removing hydrogen atom without neighbors
[02:13:07] WARNING: not removing hydrogen atom without neighbors
[02:13:07] WARNING: not removing hydrogen atom without neighbors
[02:13:07] WARNING: not removing hydrogen atom without neighbors
[02:13:07] WARNING: not removing hydrogen atom without neighbors
[02:13:07] WARNING: not removing hydrogen atom without neighbors
[02:13:07] WARNING: not removing hydrogen atom without neighbors
[02:13:07] WARNING: not removing hydrogen atom without neighbors
[02:13:07] WARNING: not removing hydrogen atom without neighbors
[02:13:07] WARNING: not removing hydrogen atom without neighbors
[02:13:07] WARNING: not r

In [192]:
def generate(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_NumRotatableBonds = Descriptors.NumRotatableBonds(mol)
           
        row = np.array([desc_MolLogP,
                        desc_MolWt,
                        desc_NumRotatableBonds])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MolLogP","MolWt","NumRotatableBonds"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [193]:
df = generate(data.SMILES)
df

[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not removing hydrogen atom without neighbors
[02:13:54] WARNING: not r

,MolLogP,MolWt,NumRotatableBonds
0,3.95810,392.510,17.0
1,2.40550,169.183,0.0
2,2.15250,140.569,1.0
3,8.11610,756.226,10.0
4,2.48540,422.525,12.0
...,...,...,...
9977,2.61700,264.369,8.0
9978,-0.21440,444.440,2.0
9979,2.82402,150.221,1.0
9980,5.09308,454.611,13.0


In [194]:
m = Chem.MolFromSmiles('COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21') 

In [195]:
aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
aromatic_atoms

[False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True]

In [196]:
def AromaticAtoms(m):
  aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
  aa_count = []
  for i in aromatic_atoms:
    if i==True:
      aa_count.append(1)
  sum_aa_count = sum(aa_count)
  return sum_aa_count

In [197]:
AromaticAtoms(m)

19

In [198]:
desc_AromaticAtoms = [AromaticAtoms(element) for element in mol_list]
desc_AromaticAtoms

[0,
 10,
 6,
 36,
 12,
 6,
 0,
 0,
 12,
 12,
 6,
 6,
 36,
 6,
 6,
 0,
 6,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 6,
 0,
 0,
 0,
 0,
 6,
 16,
 0,
 0,
 12,
 6,
 6,
 0,
 0,
 6,
 0,
 28,
 12,
 30,
 12,
 9,
 0,
 28,
 28,
 0,
 0,
 6,
 0,
 14,
 6,
 6,
 6,
 0,
 0,
 0,
 12,
 0,
 12,
 0,
 0,
 0,
 6,
 0,
 0,
 24,
 0,
 6,
 6,
 0,
 12,
 12,
 24,
 0,
 18,
 6,
 0,
 12,
 0,
 0,
 0,
 10,
 0,
 30,
 0,
 24,
 0,
 0,
 6,
 26,
 0,
 0,
 0,
 18,
 12,
 0,
 0,
 0,
 0,
 0,
 6,
 24,
 12,
 0,
 0,
 0,
 6,
 0,
 6,
 0,
 0,
 0,
 12,
 0,
 0,
 0,
 18,
 16,
 6,
 0,
 12,
 0,
 24,
 6,
 12,
 0,
 24,
 0,
 6,
 6,
 0,
 26,
 6,
 0,
 26,
 0,
 0,
 6,
 0,
 0,
 9,
 6,
 6,
 6,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 12,
 0,
 0,
 18,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 6,
 0,
 0,
 54,
 6,
 0,
 0,
 12,
 6,
 6,
 6,
 6,
 0,
 0,
 0,
 0,
 12,
 0,
 12,
 6,
 12,
 0,
 0,
 0,
 22,
 0,
 12,
 18,
 6,
 12,
 0,
 6,
 6,
 0,
 0,
 6,
 0,
 36,
 6,
 0,
 6,
 0,
 12,
 0,
 0,
 0,
 6,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 6,
 6,
 10,
 0,
 6,
 38,


In [199]:
#single molecule
m = Chem.MolFromSmiles('COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21')
Descriptors.HeavyAtomCount(m)

34

In [200]:
#all dataset
desc_HeavyAtomCount = [Descriptors.HeavyAtomCount(element) for element in mol_list]
desc_HeavyAtomCount

[23,
 13,
 9,
 53,
 31,
 9,
 12,
 27,
 16,
 25,
 11,
 14,
 54,
 15,
 18,
 11,
 14,
 35,
 11,
 16,
 14,
 10,
 10,
 7,
 13,
 31,
 10,
 10,
 27,
 18,
 2,
 14,
 35,
 13,
 35,
 27,
 16,
 12,
 1,
 14,
 11,
 9,
 69,
 26,
 49,
 28,
 16,
 11,
 63,
 58,
 12,
 18,
 21,
 21,
 15,
 11,
 14,
 13,
 14,
 12,
 10,
 20,
 29,
 21,
 63,
 13,
 5,
 9,
 16,
 26,
 42,
 13,
 15,
 24,
 2,
 16,
 25,
 42,
 10,
 21,
 14,
 15,
 28,
 10,
 9,
 7,
 11,
 10,
 54,
 3,
 52,
 14,
 5,
 16,
 42,
 12,
 16,
 27,
 33,
 16,
 12,
 13,
 8,
 13,
 14,
 20,
 46,
 17,
 19,
 19,
 156,
 21,
 28,
 18,
 22,
 12,
 45,
 16,
 58,
 19,
 64,
 18,
 16,
 20,
 14,
 23,
 34,
 40,
 13,
 22,
 63,
 56,
 3,
 19,
 13,
 37,
 44,
 10,
 4,
 42,
 14,
 10,
 11,
 1,
 2,
 13,
 21,
 10,
 11,
 7,
 11,
 3,
 23,
 7,
 11,
 13,
 9,
 16,
 20,
 22,
 31,
 25,
 32,
 22,
 26,
 6,
 13,
 43,
 14,
 3,
 24,
 11,
 12,
 13,
 5,
 24,
 15,
 13,
 12,
 100,
 11,
 12,
 16,
 27,
 13,
 14,
 11,
 13,
 10,
 6,
 16,
 14,
 34,
 9,
 20,
 14,
 17,
 17,
 5,
 1,
 28,
 16,
 13,
 35,
 15,
 3

In [201]:
m = Chem.MolFromSmiles('COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21')
AromaticAtoms(m)/Descriptors.HeavyAtomCount(m)

0.5588235294117647

In [202]:
desc_AromaticProportion = [AromaticAtoms(element)/Descriptors.HeavyAtomCount(element) for element in mol_list]
desc_AromaticProportion

[0.0,
 0.7692307692307693,
 0.6666666666666666,
 0.6792452830188679,
 0.3870967741935484,
 0.6666666666666666,
 0.0,
 0.0,
 0.75,
 0.48,
 0.5454545454545454,
 0.42857142857142855,
 0.6666666666666666,
 0.4,
 0.3333333333333333,
 0.0,
 0.42857142857142855,
 0.0,
 0.5454545454545454,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.46153846153846156,
 0.0,
 0.6,
 0.0,
 0.0,
 0.0,
 0.0,
 0.42857142857142855,
 0.45714285714285713,
 0.0,
 0.0,
 0.4444444444444444,
 0.375,
 0.5,
 0.0,
 0.0,
 0.5454545454545454,
 0.0,
 0.4057971014492754,
 0.46153846153846156,
 0.6122448979591837,
 0.42857142857142855,
 0.5625,
 0.0,
 0.4444444444444444,
 0.4827586206896552,
 0.0,
 0.0,
 0.2857142857142857,
 0.0,
 0.9333333333333333,
 0.5454545454545454,
 0.42857142857142855,
 0.46153846153846156,
 0.0,
 0.0,
 0.0,
 0.6,
 0.0,
 0.5714285714285714,
 0.0,
 0.0,
 0.0,
 0.6666666666666666,
 0.0,
 0.0,
 0.5714285714285714,
 0.0,
 0.4,
 0.25,
 0.0,
 0.75,
 0.48,
 0.5714285714285714,
 0.0,
 0.8571428571428571,
 0.42857142857142855,

In [203]:
df_desc_AromaticProportion = pd.DataFrame(desc_AromaticProportion, columns=['AromaticProportion'])
df_desc_AromaticProportion

,AromaticProportion
0,0.000000
1,0.769231
2,0.666667
3,0.679245
4,0.387097
...,...
9977,0.315789
9978,0.187500
9979,0.545455
9980,0.363636


In [204]:
df

,MolLogP,MolWt,NumRotatableBonds
0,3.95810,392.510,17.0
1,2.40550,169.183,0.0
2,2.15250,140.569,1.0
3,8.11610,756.226,10.0
4,2.48540,422.525,12.0
...,...,...,...
9977,2.61700,264.369,8.0
9978,-0.21440,444.440,2.0
9979,2.82402,150.221,1.0
9980,5.09308,454.611,13.0


In [205]:
df_desc_AromaticProportion


,AromaticProportion
0,0.000000
1,0.769231
2,0.666667
3,0.679245
4,0.387097
...,...
9977,0.315789
9978,0.187500
9979,0.545455
9980,0.363636


In [207]:
X = pd.concat([df,df_desc_AromaticProportion], axis=1)
X

,MolLogP,MolWt,NumRotatableBonds,AromaticProportion
0,3.95810,392.510,17.0,0.000000
1,2.40550,169.183,0.0,0.769231
2,2.15250,140.569,1.0,0.666667
3,8.11610,756.226,10.0,0.679245
4,2.48540,422.525,12.0,0.387097
...,...,...,...,...
9977,2.61700,264.369,8.0,0.315789
9978,-0.21440,444.440,2.0,0.187500
9979,2.82402,150.221,1.0,0.545455
9980,5.09308,454.611,13.0,0.363636


In [209]:
data.head()

,ID,Name,InChI,InChIKey,SMILES,Solubility,SD,Ocurrences,Group,MolWt,...,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT
0,A-3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1,G1,392.510,...,17.0,142.0,0.0,0.0,0.0,0.0,0.00,158.520601,0.000000e+00,210.377334
1,A-4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1,G1,169.183,...,0.0,62.0,2.0,0.0,1.0,3.0,29.10,75.183563,2.582996e+00,511.229248
2,A-5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1,G1,140.569,...,1.0,46.0,1.0,0.0,0.0,1.0,17.07,58.261134,3.009782e+00,202.661065
3,A-8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.0,1,G1,756.226,...,10.0,264.0,6.0,0.0,0.0,6.0,120.72,323.755434,2.322963e-07,1964.648666
4,A-9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1,G1,422.525,...,12.0,164.0,2.0,4.0,4.0,6.0,56.60,183.183268,1.084427e+00,769.899934


In [210]:
Y = data.iloc[:,1]
Y

0              N,N,N-trimethyloctadecan-1-aminium bromide
1                                Benzo[cd]indol-2(1H)-one
2                                    4-chlorobenzaldehyde
3       zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...
4       4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...
                              ...                        
9977                                           tetracaine
9978                                         tetracycline
9979                                               thymol
9980                                            verapamil
9981                                             warfarin
Name: Name, Length: 9982, dtype: object

In [211]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [213]:
X_t2 = X_train.select_dtypes(['string'])
print (X_t2)

X_train[X_t2.columns] = X_t2.apply(lambda x: x.str.strip())
print (X_train)

Empty DataFrame
Columns: []
Index: [8815, 7175, 4956, 6916, 9462, 8578, 2474, 8763, 6627, 195, 7909, 7772, 2699, 6140, 589, 607, 1740, 5952, 5614, 6802, 5666, 1848, 1551, 7681, 7157, 1100, 9278, 2420, 8510, 4931, 3724, 5462, 1835, 2877, 2646, 4346, 3361, 3928, 477, 9197, 1851, 4163, 2026, 7843, 3245, 7163, 7729, 4283, 2635, 7247, 903, 9647, 3377, 1070, 8332, 3758, 4090, 4800, 541, 5826, 475, 2336, 7413, 5509, 4459, 6325, 6941, 4406, 5525, 4435, 9445, 8996, 802, 910, 6012, 9377, 3894, 5831, 2289, 5951, 2141, 5692, 7743, 3328, 7361, 6162, 4401, 5111, 7310, 929, 6602, 8768, 6303, 4100, 652, 6593, 2395, 3624, 62, 2163, ...]

[7985 rows x 0 columns]
      MolLogP    MolWt  NumRotatableBonds  AromaticProportion
8815   1.1278  131.175                1.0            0.000000
7175   1.8030  229.715                5.0            0.400000
4956  -0.8295  174.200                4.0            0.000000
6916   4.6401  314.821                6.0            0.375000
9462   3.0345  358.467               

In [214]:
Y_train = Y_train.to_frame()


In [215]:
Y_t2 = Y_train.select_dtypes(['string'])
print (Y_t2)

Y_train[Y_t2.columns] = Y_t2.apply(lambda x: x.str.strip())
print (Y_train)

Empty DataFrame
Columns: []
Index: [8815, 7175, 4956, 6916, 9462, 8578, 2474, 8763, 6627, 195, 7909, 7772, 2699, 6140, 589, 607, 1740, 5952, 5614, 6802, 5666, 1848, 1551, 7681, 7157, 1100, 9278, 2420, 8510, 4931, 3724, 5462, 1835, 2877, 2646, 4346, 3361, 3928, 477, 9197, 1851, 4163, 2026, 7843, 3245, 7163, 7729, 4283, 2635, 7247, 903, 9647, 3377, 1070, 8332, 3758, 4090, 4800, 541, 5826, 475, 2336, 7413, 5509, 4459, 6325, 6941, 4406, 5525, 4435, 9445, 8996, 802, 910, 6012, 9377, 3894, 5831, 2289, 5951, 2141, 5692, 7743, 3328, 7361, 6162, 4401, 5111, 7310, 929, 6602, 8768, 6303, 4100, 652, 6593, 2395, 3624, 62, 2163, ...]

[7985 rows x 0 columns]
                                                   Name
8815                                tert-Amyl_Carbamate
7175                                         trietazine
4956                                l-valine, n-glycyl-
6916                             carbophenothion-methyl
9462  4-amino-~{N}-(1,11,11-trimethyl-3,5-diazatricy...
...        

In [219]:
model = linear_model.LinearRegression()
model.fit(X_train, Y_train)
     

ValueError: could not convert string to float: 'tert-Amyl_Carbamate'